<font color= "green">Here the mother computer acts as a hub, collecting data from FED and then sending commands or events to the Arduino board.
Arduino receives those event markers from the mother computer and translates them into digital pulses for the TDT system</font>

## The data flow that we need to sync FEDs with TDT:
### FED3 to Computer: The FED3 devices send behavioral event data to the computer through their respective serial connections.
### Computer to Arduino: The computer processes this data and forwards relevant event information to the Arduino through a separate serial connection.
### Arduino to TDT System: The Arduino generates digital pulses or signals in response to the data it receives. These pulses are sent to the TDT system to mark the timing of behavioral events, enabling synchronization with neural recordings.

In [ ]:
#example of the code we can use to flash our arduino board

//int pulsePin = 10; // Use pin 10 for the LED or pulse output
//
//void setup() {
//  Serial.begin(115200);
//  pinMode(pulsePin, OUTPUT); // Set pin 10 as an output
//}
//
//void loop() {
//  if (Serial.available() > 0) {
//    String data = Serial.readStringUntil('\n');
//    
//    // Generate a pulse when data is received
//    digitalWrite(pulsePin, HIGH);  // Turn the LED on or send a pulse
//    delay(10);                     // 10 ms pulse duration
//    digitalWrite(pulsePin, LOW);   // Turn the LED off or end the pulse
//    
//    Serial.print("Received: ");
//    Serial.println(data);  // Print the received data to the Serial Monitor
//  }
//}



int pulsePin = 10; // Digital pin for output pulse to TDT

void setup() {
  Serial.begin(115200); // Initialize serial communication at 115200 baud rate
  pinMode(pulsePin, OUTPUT); // Set pin 10 as an output for the pulse
}

void loop() {
  if (Serial.available() > 0) { 
    // Check if data is available to read from the serial port
    char incomingByte = Serial.read(); // Read the incoming byte
    
    // Generate a pulse on pin 10 to send a signal to the TDT system
    digitalWrite(pulsePin, HIGH);  // Send a HIGH signal (pulse) to TDT
    delay(10);                     // 10 ms pulse duration (adjust if needed)
    digitalWrite(pulsePin, LOW);   // End the pulse (set pin 10 to LOW)

    // (Optional) Print the received byte for debugging
    Serial.print("Received: ");
    Serial.println(incomingByte);
  }
}


In [1]:
# this code should send data to Google spreadsheet and at the same time to Arduino board

import serial
import threading
import datetime  
import gspread
from google.oauth2.service_account import Credentials

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Setup Google Sheets
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Path to your downloaded JSON file
CREDS_FILE = r"C:\Users\hta031\Github\HOME_PHOTOMETRY\scripts\Python_codes\homephotometry-102601e8b10f.json"

creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
client = gspread.authorize(creds)

# Google Spreadsheet ID
SPREADSHEET_ID = "1oybqWp_7b9_oiR-a1Xy0YLw8LwvGfqtmSz2lYfEzrBk"

def get_or_create_worksheet(spreadsheet, title):
    try:
        # Try to open the worksheet by title
        sheet = spreadsheet.worksheet(title)
        print(f"Worksheet '{title}' found.")
    except gspread.exceptions.WorksheetNotFound:
        # If the worksheet is not found, create it
        print(f"Worksheet '{title}' not found. Creating a new one.")
        sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
        sheet.append_row(column_headers)
    return sheet

def send_to_arduino(data, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    arduino_ser.write(data.encode('utf-8'))
    print(f"Sent to Arduino: {data}")

def read_from_port(ser, worksheet_name, arduino_ser):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        # to sync all the FED units with each other, I tend not to rely on FED clock,
        # Ignore the first field (timestamp from FED device) and use the computer's timestamp
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        # Assuming the data matches the order of the remaining column_headers
        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Append the row to Google Sheet
            sheet.append_row([timestamp] + data_list)
            
            # Send relevant data to Arduino
            event_type = data_list[9]  # Example: using the 10th field as the event identifier
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM12"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    worksheet_name = f"Port_{port}"  # Create a unique worksheet name for each port
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, worksheet_name, arduino_ser)).start()


Worksheet 'Port_COM12' not found. Creating a new one.


# I noticed a delay between FEd logging the data and LED blinking on arduino, here I am trying to get around it by sending the pulse to Arduino first and then to spreadsheet

In [1]:
import serial
import threading
import datetime  
import gspread
from google.oauth2.service_account import Credentials

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Setup Google Sheets
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Path to your downloaded JSON file
CREDS_FILE = r"C:\Users\hta031\Github\HOME_PHOTOMETRY\scripts\Python_codes\homephotometry-102601e8b10f.json"

creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
client = gspread.authorize(creds)

# Google Spreadsheet ID
SPREADSHEET_ID = "1oybqWp_7b9_oiR-a1Xy0YLw8LwvGfqtmSz2lYfEzrBk"

def get_or_create_worksheet(spreadsheet, title):
    try:
        # Try to open the worksheet by title
        sheet = spreadsheet.worksheet(title)
        print(f"Worksheet '{title}' found.")
    except gspread.exceptions.WorksheetNotFound:
        # If the worksheet is not found, create it
        print(f"Worksheet '{title}' not found. Creating a new one.")
        sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
        sheet.append_row(column_headers)
    return sheet

def send_to_arduino(data, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    arduino_ser.write(data.encode('utf-8'))
    print(f"Sent to Arduino: {data}")

def log_to_google_sheet(sheet, timestamp, data_list):
    """
    Append the data to Google Sheets after sending it to Arduino.
    """
    sheet.append_row([timestamp] + data_list)

def read_from_port(ser, worksheet_name, arduino_ser):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Send relevant data to Arduino first
            event_type = data_list[9]  # Example: using the 10th field as the event identifier
            send_to_arduino(event_type, arduino_ser)
            
            # Then log the data to Google Sheets
            log_to_google_sheet(sheet, timestamp, data_list)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM12","COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    worksheet_name = f"Port_{port}"  # Create a unique worksheet name for each port
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, worksheet_name, arduino_ser)).start()


Worksheet 'Port_COM16' not found. Creating a new one.
Worksheet 'Port_COM12' not found. Creating a new one.
Data from COM12: 8/22/2024 10:20:37,22.85,52.42,1.16.3,FR1,8,4.19,NaN,1,Right,Left,4,65,4,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/22/2024 10:14:05,22.14,54.37,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.01
Sent to Arduino: Left
Data from COM16: 8/22/2024 10:14:07,22.16,54.39,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,2,0,0,NaN,NaN,0.03
Sent to Arduino: Left
Data from COM16: 8/22/2024 10:14:57,22.33,53.67,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,3,0,0,NaN,NaN,0.26
Sent to Arduino: Left
Data from COM12: 8/22/2024 10:21:30,22.85,52.37,1.16.3,FR1,8,4.19,NaN,1,Right,Left,4,66,4,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/22/2024 10:14:58,22.34,53.67,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,4,0,0,NaN,NaN,0.06
Sent to Arduino: Left
Data from COM12: 8/22/2024 10:21:33,22.87,52.37,1.16.3,FR1,8,4.20,NaN,1,Right,Left,4,67,4,0,NaN,NaN,0.00
Sent to Arduino: Left
Dat

# to make separate event signals to the TDT we need to adjust the python script that gets FED readout and also how Arduino signals TDT

In [ ]:

# I wil replace this snipet with the orginal send_to_arduino function
def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
    elif event_type == "Pellet":
        arduino_ser.write(b'P')  # Pellet intake event
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
    print(f"Sent to Arduino: {event_type}")


#then I will flash the arduino code with this script 
int pulsePin = 10; // Pin for output pulse

void setup() {
  Serial.begin(115200);
  pinMode(pulsePin, OUTPUT); // Set pin 10 as an output
}

void loop() {
  if (Serial.available() > 0) {
    char eventType = Serial.read(); // Read the event type identifier

    switch(eventType) {
      case 'R': // Right poke event
        generatePulse(1); // Single pulse
        break;
      case 'L': // Left poke event
        generatePulse(2); // Double pulse
        break;
      case 'P': // Pellet intake event
        generatePulse(3); // Triple pulse
        break;
      case 'W': // Left with pellet event
        generatePulse(4); // Quadruple pulse
        break;
      case 'Q': // Right with pellet event
        generatePulse(5); // Quintuple pulse
        break;
    }
  }
}

void generatePulse(int pulseCount) {
  for (int i = 0; i < pulseCount; i++) {
    digitalWrite(pulsePin, HIGH);  // Send a pulse
    delay(10);                     // 10 ms pulse duration
    digitalWrite(pulsePin, LOW);   // End the pulse
    delay(100);                    // Wait 100 ms between pulses
  }
}



# code below testing distinct TTL signals
# code below is tested and sends TTL signals according to the behavioural event
# This code requires internet connection since it needs to log the data on google spreadsheet as well

In [2]:
import serial
import threading
import datetime  
import gspread
from google.oauth2.service_account import Credentials

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Setup Google Sheets
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Path to your downloaded JSON file
CREDS_FILE = r"C:\Users\hta031\Github\HOME_PHOTOMETRY\scripts\Python_codes\homephotometry-102601e8b10f.json"

creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
client = gspread.authorize(creds)

# Google Spreadsheet ID
SPREADSHEET_ID = "1oybqWp_7b9_oiR-a1Xy0YLw8LwvGfqtmSz2lYfEzrBk"

def get_or_create_worksheet(spreadsheet, title):
    try:
        # Try to open the worksheet by title
        sheet = spreadsheet.worksheet(title)
        print(f"Worksheet '{title}' found.")
    except gspread.exceptions.WorksheetNotFound:
        # If the worksheet is not found, create it
        print(f"Worksheet '{title}' not found. Creating a new one.")
        sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
        sheet.append_row(column_headers)
    return sheet

def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
    elif event_type == "Pellet":
        arduino_ser.write(b'P')  # Pellet intake event
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
    print(f"Sent to Arduino: {event_type}")

def log_to_google_sheet(sheet, timestamp, data_list):
    """
    Append the data to Google Sheets after sending it to Arduino.
    """
    sheet.append_row([timestamp] + data_list)

def read_from_port(ser, worksheet_name, arduino_ser):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Send relevant data to Arduino first
            event_type = data_list[9]  # Example: using the 10th field as the event identifier
            send_to_arduino(event_type, arduino_ser)
            
            # Then log the data to Google Sheets
            log_to_google_sheet(sheet, timestamp, data_list)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    worksheet_name = f"Port_{port}"  # Create a unique worksheet name for each port
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, worksheet_name, arduino_ser)).start()


SerialException: could not open port 'COM38': PermissionError(13, 'Access is denied.', None, 5)

# code below is the same code as above, but does not send data to the spreadsheet so should not need internet connection

In [1]:
import serial
import threading
import datetime

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
    elif event_type == "Pellet":
        arduino_ser.write(b'P')  # Pellet intake event
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
    print(f"Sent to Arduino: {event_type}")

def read_from_port(ser, arduino_ser):
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Send relevant data to Arduino
            event_type = data_list[9]  # "Event" field contains event type
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, arduino_ser)).start()


Data from COM16: 8/23/2024 11:10:52,24.10,50.65,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:10:56,24.10,50.66,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,2,0,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:10:57,24.11,50.63,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,3,0,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:10:58,24.10,50.65,1.16.3,FR1,20,4.19,NaN,1,Right,Left,0,4,0,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:11:00,24.12,50.66,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,5,0,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:11:01,24.11,50.67,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,6,0,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:11:02,24.12,50.64,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,7,0,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:11:03,24.12,50.63,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,8,0,0,NaN,NaN,0.00
Sent t

# Cell below just some minor adjustments to the previuse code before testing the new Arduino script where each event is signalled via a separate pin

In [1]:
import serial
import threading
import datetime

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
    elif event_type == "Pellet":
        arduino_ser.write(b'P')  # Pellet intake event
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
    else:
        print(f"Unknown event type: {event_type}")
    print(f"Sent to Arduino: {event_type}")

def read_from_port(ser, arduino_ser):
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Send relevant data to Arduino
            event_type = data_list[9]  # "Event" field contains event type
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, arduino_ser)).start()


Data from COM16: 8/23/2024 11:54:09,24.35,51.06,1.16.3,FR1,20,4.20,NaN,1,Right,Left,2,12,2,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:54:13,24.34,51.06,1.16.3,FR1,20,4.20,NaN,1,Left,Left,3,12,2,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:54:16,24.35,51.06,1.16.3,FR1,20,4.20,2,1,Pellet,Left,3,12,3,0,1.45,2573,NaN
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:54:19,24.35,51.05,1.16.3,FR1,20,4.19,NaN,1,Right,Left,3,13,3,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:54:20,24.35,51.06,1.16.3,FR1,20,4.20,NaN,1,Right,Left,3,14,3,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:54:20,24.37,51.04,1.16.3,FR1,20,4.19,NaN,1,Right,Left,3,15,3,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:54:21,24.37,51.03,1.16.3,FR1,20,4.19,NaN,1,Right,Left,3,16,3,0,NaN,NaN,0.00
Sent to Arduino: Left
Data from COM16: 8/23/2024 11:54:56,24.35,50.99,1.16.3,FR1,20,4.20,NaN,1,Right,Left,3,17,3,0,NaN,NaN,0.00

# code below the same as above with a bit more debuging statements 

In [1]:

import serial
import threading
import datetime

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    event_type = event_type.strip()  # Strip whitespace but keep the event type uppercase
    print(f"Normalized event type: '{event_type}'")

    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
        print("Sending 'R' for Right poke event")
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
        print("Sending 'L' for Left poke event")
    elif event_type == "Pellet":
        arduino_ser.write(b'P')  # Pellet intake event
        print("Sending 'P' for Pellet intake event")
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
        print("Sending 'W' for LeftWithPellet event")
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
        print("Sending 'Q' for RightWithPellet event")
    else:
        print(f"Unknown event type: '{event_type}'")
    print(f"Sent to Arduino: {event_type}")

def read_from_port(ser, arduino_ser):
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Extract and print the event type
            event_type = data_list[8]  # "Event" field contains event type
            print(f"Extracted event type: '{event_type}'")
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, arduino_ser)).start()


Data from COM16: 8/23/2024 17:43:36,24.32,53.98,1.16.3,FR1,20,4.19,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.00
Extracted event type: 'Right'
Normalized event type: 'Right'
Sending 'R' for Right poke event
Sent to Arduino: Right
Data from COM16: 8/23/2024 17:43:40,24.32,53.99,1.16.3,FR1,20,4.20,NaN,1,Left,Left,1,1,0,0,NaN,NaN,0.00
Extracted event type: 'Left'
Normalized event type: 'Left'
Sending 'L' for Left poke event
Sent to Arduino: Left
Data from COM16: 8/23/2024 17:43:46,24.31,53.97,1.16.3,FR1,20,4.19,NaN,1,PelletInWell,Left,1,1,0,0,NaN,NaN,NaN
Extracted event type: 'PelletInWell'
Normalized event type: 'PelletInWell'
Unknown event type: 'PelletInWell'
Sent to Arduino: PelletInWell
Data from COM16: 8/23/2024 17:43:52,24.31,53.94,1.16.3,FR1,20,4.20,4,1,Pellet,Left,1,1,1,0,6.00,NaN,NaN
Extracted event type: 'Pellet'
Normalized event type: 'Pellet'
Sending 'P' for Pellet intake event
Sent to Arduino: Pellet
Data from COM16: 8/23/2024 17:43:56,24.29,53.93,1.16.3,FR1,20,4.20,NaN,1,Right,Left

# Cell below testing the PelletInWell signal -- I have updated the library to also log PelletInWell when  the pellet is dispenesed but not taken by the mouse

In [1]:
import serial
import threading
import datetime

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    event_type = event_type.strip()  # Strip whitespace but keep the event type uppercase
    print(f"Normalized event type: '{event_type}'")

    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
        print("Sending 'R' for Right poke event")
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
        print("Sending 'L' for Left poke event")
    elif event_type in ["Pellet", "PelletInWell"]:
        arduino_ser.write(b'P')  # Pellet intake or pellet in well event
        print(f"Sending 'P' for {event_type} event")
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
        print("Sending 'W' for LeftWithPellet event")
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
        print("Sending 'Q' for RightWithPellet event")
    else:
        print(f"Unknown event type: '{event_type}'")
    print(f"Sent to Arduino: {event_type}")

def read_from_port(ser, arduino_ser):
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Extract and print the event type
            event_type = data_list[8]  # "Event" field contains event type
            print(f"Extracted event type: '{event_type}'")
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM39"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, arduino_ser)).start()





##########################
####Arduino should be flashed with this code

# int leftPokePin = 9;           // Pin for left poke event
# int leftWithPelletPin = 10;    // Pin for left poke with pellet event
# int rightPokePin = 11;         // Pin for right poke event
# int rightWithPelletPin = 12;   // Pin for right poke with pellet event
# int pelletIntakePin = 13;      // Pin for pellet intake event
# int pelletInWellPin = 14;      // Pin for pellet in well event

# void setup() {
#   Serial.begin(115200);
#   pinMode(leftPokePin, OUTPUT);         
#   pinMode(leftWithPelletPin, OUTPUT);   
#   pinMode(rightPokePin, OUTPUT);        
#   pinMode(rightWithPelletPin, OUTPUT);  
#   pinMode(pelletIntakePin, OUTPUT);     
#   pinMode(pelletInWellPin, OUTPUT);     
# }

# void loop() {
#   if (Serial.available() > 0) {
#     char eventType = Serial.read(); // Read the event type identifier
#     Serial.print("Received event type: ");
#     Serial.println(eventType); // Print the received event type for debugging

#     if (eventType == 'L') {
#         digitalWrite(leftPokePin, HIGH);   
#         delay(10);                       
#         digitalWrite(leftPokePin, LOW);    
#         Serial.println("Left poke LED triggered");
#     } 
#     else if (eventType == 'W') {
#         digitalWrite(leftWithPelletPin, HIGH);  
#         delay(10);                            
#         digitalWrite(leftWithPelletPin, LOW);   
#         Serial.println("Left with pellet LED triggered");
#     } 
#     else if (eventType == 'R') {
#         digitalWrite(rightPokePin, HIGH);  
#         delay(10);                       
#         digitalWrite(rightPokePin, LOW);   
#         Serial.println("Right poke LED triggered");
#     } 
#     else if (eventType == 'Q') {
#         digitalWrite(rightWithPelletPin, HIGH);  
#         delay(10);                             
#         digitalWrite(rightWithPelletPin, LOW);   
#         Serial.println("Right with pellet LED triggered");
#     }
#     else if (eventType == 'P') {
#         digitalWrite(pelletIntakePin, HIGH);  
#         delay(10);                          
#         digitalWrite(pelletIntakePin, LOW);   
#         Serial.println("Pellet intake LED triggered");
#     }
#     else if (eventType == 'I') {  // Handle PelletInWell event
#         digitalWrite(pelletInWellPin, HIGH);  
#         delay(10);                          
#         digitalWrite(pelletInWellPin, LOW);   
#         Serial.println("Pellet in well LED triggered");
#     }
#   }
# }

####################################### to signal Pellet intake and Pellet in well with the same pin########
# int leftPokePin = 9;           // Pin for left poke event
# int leftWithPelletPin = 10;    // Pin for left poke with pellet event
# int rightPokePin = 11;         // Pin for right poke event
# int rightWithPelletPin = 12;   // Pin for right poke with pellet event
# int pelletPin = 13;            // Pin for both pellet in well and pellet intake events

# bool pelletInWell = false;  // State variable to track if pellet is in the well

# void setup() {
#   Serial.begin(115200);

#   // Setup pins as outputs
#   pinMode(leftPokePin, OUTPUT);         
#   pinMode(leftWithPelletPin, OUTPUT);   
#   pinMode(rightPokePin, OUTPUT);        
#   pinMode(rightWithPelletPin, OUTPUT);  
#   pinMode(pelletPin, OUTPUT);     

#   // Ensure all pins start LOW
#   digitalWrite(leftPokePin, LOW);
#   digitalWrite(leftWithPelletPin, LOW);
#   digitalWrite(rightPokePin, LOW);
#   digitalWrite(rightWithPelletPin, LOW);
#   digitalWrite(pelletPin, LOW);
# }

# void loop() {
#   if (Serial.available() > 0) {
#     char eventType = Serial.read(); // Read the event type identifier
#     Serial.print("Received event type: ");
#     Serial.println(eventType); // Print the received event type for debugging

#     switch (eventType) {
#       case 'L': // Left poke event
#         digitalWrite(leftPokePin, HIGH);
#         delay(10);
#         digitalWrite(leftPokePin, LOW);
#         Serial.println("Left poke LED triggered");
#         break;

#       case 'W': // Left with pellet event
#         digitalWrite(leftWithPelletPin, HIGH);
#         delay(10);
#         digitalWrite(leftWithPelletPin, LOW);
#         Serial.println("Left with pellet LED triggered");
#         break;

#       case 'R': // Right poke event
#         digitalWrite(rightPokePin, HIGH);
#         delay(10);
#         digitalWrite(rightPokePin, LOW);
#         Serial.println("Right poke LED triggered");
#         break;

#       case 'Q': // Right with pellet event
#         digitalWrite(rightWithPelletPin, HIGH);
#         delay(10);
#         digitalWrite(rightWithPelletPin, LOW);
#         Serial.println("Right with pellet LED triggered");
#         break;

#       case 'P': // Pellet in well or pellet intake event
#         if (pelletInWell) {
#           // Pellet is taken
#           digitalWrite(pelletPin, LOW);   // Turn off the signal briefly
#           delay(10);                      // Short delay
#           pelletInWell = false;           // Update state
#           Serial.println("Pellet taken: Signal turned OFF briefly");
#         } else {
#           // Pellet is in the well
#           digitalWrite(pelletPin, HIGH);  // Keep the signal on
#           pelletInWell = true;            // Update state
#           Serial.println("Pellet in well: Signal turned ON");
#         }
#         break;

#       default:
#         Serial.println("Unknown event type received");
#         break;
#     }
#   }
# }




Data from COM16: 8/26/2024 16:42:48,23.43,43.08,1.16.3,FR1,20,4.20,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.00
Extracted event type: 'Right'
Normalized event type: 'Right'
Sending 'R' for Right poke event
Sent to Arduino: Right
Data from COM16: 8/26/2024 16:42:52,23.44,43.09,1.16.3,FR1,20,4.20,NaN,1,Left,Left,1,1,0,0,NaN,NaN,0.00
Extracted event type: 'Left'
Normalized event type: 'Left'
Sending 'L' for Left poke event
Sent to Arduino: Left
Data from COM16: 8/26/2024 16:42:55,23.42,43.08,1.16.3,FR1,20,4.20,NaN,1,PelletInWell,Left,1,1,0,0,NaN,NaN,NaN
Extracted event type: 'PelletInWell'
Normalized event type: 'PelletInWell'
Sending 'P' for PelletInWell event
Sent to Arduino: PelletInWell
Data from COM16: 8/26/2024 16:42:58,23.43,43.11,1.16.3,FR1,20,4.20,2,1,Pellet,Left,1,1,1,0,3.19,NaN,NaN
Extracted event type: 'Pellet'
Normalized event type: 'Pellet'
Sending 'P' for Pellet event
Sent to Arduino: Pellet
Data from COM16: 8/26/2024 16:43:01,23.43,43.12,1.16.3,FR1,20,4.20,NaN,1,Right,Left,1,2,1,

Exception in thread Thread-5 (read_from_port):
Traceback (most recent call last):
  File "C:\Users\hta031\AppData\Local\miniconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\hta031\AppData\Local\miniconda3\Lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\hta031\AppData\Local\miniconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hta031\AppData\Local\Temp\ipykernel_43328\139459181.py", line 42, in read_from_port
  File "C:\Users\hta031\AppData\Local\miniconda3\Lib\site-packages\serial\serialwin32.py", line 275, in read
    raise SerialException("ClearCommError failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))


# to connect multiple FEDs with their own TTL signals, let's try the code below

In [1]:
def send_to_arduino(fed_id, event_type, arduino_ser):
    """
    Send the event data and FED identifier to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    command = f"{fed_id}_{event_type}"  # Create a unique command for each FED and event
    arduino_ser.write(command.encode('utf-8'))
    print(f"Sent to Arduino: {command}")

def read_from_port(ser, worksheet_name, arduino_ser, fed_id):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            event_type = data_list[9]  # "Event" field contains event type
            send_to_arduino(fed_id, event_type, arduino_ser)  # Send FED ID and event type to Arduino
            log_to_google_sheet(sheet, timestamp, data_list)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")


# and then I will need to flash the arduino with this script
int pulsePin = 10; // Pin for output pulse

void setup() {
  Serial.begin(115200);
  pinMode(pulsePin, OUTPUT); // Set pin 10 as an output
}

void loop() {
  if (Serial.available() > 0) {
    String command = Serial.readStringUntil('\n'); // Read the full command
    
    if (command == "FED1_Left") {
        generatePulse(1);  // Example: Single pulse for FED1 Left
    } else if (command == "FED1_Right") {
        generatePulse(2);  // Example: Double pulse for FED1 Right
    } else if (command == "FED2_Left") {
        generatePulse(3);  // Example: Triple pulse for FED2 Left
    } else if (command == "FED2_Right") {
        generatePulse(4);  // Example: Quadruple pulse for FED2 Right
    } // Continue with other cases
  }
}

void generatePulse(int pulseCount) {
  for (int i = 0; i < pulseCount; i++) {
    digitalWrite(pulsePin, HIGH);  // Send a pulse
    delay(10);                     // 10 ms pulse duration
    digitalWrite(pulsePin, LOW);   // End the pulse
    delay(100);                    // Wait 100 ms between pulses
  }
}


SyntaxError: invalid syntax (1957769390.py, line 31)

# DB25 Cable solution

Using a **DB25** connector can be an effective way to manage multiple signals from your Arduino to the TDT system while reducing the number of individual BNC cables required. A DB25 connector has 25 pins, allowing you to send multiple signals through a single, organized cable.

### How to Use a DB25 Connector with Arduino and TDT:

1. **DB25 Connector Overview**:
   - **25 Pins**: The DB25 connector has 25 pins, which can be used to transmit multiple signals (e.g., digital outputs from the Arduino) and ground connections through a single cable.
   - **Custom Cable**: You would typically create or purchase a custom DB25 cable that breaks out the individual wires from the DB25 connector to individual BNC connectors or directly to the TDT system.

2. **Wiring Multiple FEDs to a DB25 Connector**:
   - **Signal Assignment**: Assign each signal (event) from your Arduino to a separate pin on the DB25 connector.
   - **Shared Ground**: Use one or more pins on the DB25 connector for ground connections, which can be shared among all signals.

3. **Example Pin Assignment**:
   - Suppose you have 4 FEDs, each generating 5 events. You can assign 20 pins on the DB25 connector for these signals.
   - **Pin 1-5**: FED 1 (Left Poke, Right Poke, Pellet, LeftWithPellet, RightWithPellet)
   - **Pin 6-10**: FED 2
   - **Pin 11-15**: FED 3
   - **Pin 16-20**: FED 4
   - **Pin 21-25**: Ground or additional signals if needed.

4. **Connecting to TDT**:
   - **DB25 to BNC Breakout**: Use a DB25 to BNC breakout cable or panel, which converts the signals from the DB25 connector to individual BNC connectors that can be connected to the TDT system.
   - **Direct Connection**: If the TDT system supports DB25 inputs, you can directly connect the DB25 cable to the TDT system, reducing the need for multiple BNC connectors.

5. **Advantages**:
   - **Organized Wiring**: Reduces the clutter and complexity of using many individual BNC cables.
   - **Scalability**: Easy to add more signals or reassign pins without redoing all the wiring.
   - **Shared Ground**: Simplifies grounding by allowing multiple signals to share a common ground.

6. **Considerations**:
   - **Signal Integrity**: Ensure that the length and quality of the DB25 cable maintain signal integrity, especially if the cable runs are long.
   - **TDT Compatibility**: Confirm that your TDT system can accept the signal levels and types you're sending through the DB25 connector.

### Example Schematic:

- **Arduino Pin 9** -> **DB25 Pin 1** (Left Poke from FED 1)
- **Arduino Pin 10** -> **DB25 Pin 2** (LeftWithPellet from FED 1)
- **Arduino Pin 11** -> **DB25 Pin 3** (Right Poke from FED 1)
- **Arduino Pin 12** -> **DB25 Pin 4** (RightWithPellet from FED 1)
- **Arduino Pin 13** -> **DB25 Pin 5** (Pellet Intake from FED 1)
- Repeat for FED 2, FED 3, and FED 4.

- **Ground** -> **DB25 Pin 21-25** (Shared Ground)

### Conclusion:
Using a DB25 connector allows you to consolidate multiple signals into a single, manageable cable, making your setup with the TDT system more organized and scalable. It's especially useful when dealing with multiple FEDs or many signals, as it minimizes the number of BNC cables required and simplifies wiring.

Using an Arduino Mega and a DB25 connector is a practical approach for managing multiple FEDs. Here’s a step-by-step guide on how to set this up:

### Step 1: Adjust the Script for Multiple FEDs

1. **Modify the Python Script**: 
   - Update the Python script to handle as many FEDs as you have. Each FED should have its own set of events (e.g., Left Poke, Right Poke, etc.).
   - Each event will trigger a different pin on the Arduino Mega.

2. **Use Arduino Mega**:
   - The Arduino Mega has more digital pins (54 total), which will allow you to manage multiple FEDs without running out of pins.
   - Assign pins for each event from each FED in the Arduino code.

### Step 2: Wiring the Arduino Mega to BNC Connectors

1. **Connect Each Pin to a BNC Connector**:
   - **Signal Wire**: Connect each Arduino digital pin (e.g., pins 2 through 21) to the center conductor of a male BNC connector.
   - **Ground**: Connect the Arduino GND to the outer shield of each BNC connector. You can use multiple ground connections, or you can tie all grounds together if the BNC connectors share a common ground.

2. **BNC to DB25 Connection**:
   - **DB25 Connector**: You will need a DB25 male connector with a breakout or adapter that allows you to connect BNC connectors to specific pins on the DB25.
   - **Pin Assignment**: Assign each BNC connector’s signal wire to a specific pin on the DB25 connector. The ground wire from each BNC connector should be connected to a shared ground pin on the DB25.

### Step 3: Connecting BNCs to a DB25 Connector

1. **DB25 Breakout Board**:
   - Use a DB25 breakout board or panel that has screw terminals or solder points for each of the 25 pins. This makes it easier to connect wires from the BNC connectors.

2. **Wiring the BNC Connectors to the DB25**:
   - **Signal Wires**: Connect each signal wire from the BNC connectors to the corresponding pin on the DB25 breakout board.
   - **Ground Wires**: Connect the ground wires from all BNC connectors to one or more ground pins on the DB25 breakout board (e.g., pins 21-25).

3. **Assemble the DB25 Connector**:
   - Once all wires are connected to the DB25 breakout board, assemble the DB25 connector and secure it to the cable or panel that will connect to the TDT system.

### Example Setup for 4 FEDs:

- **FED 1**:
  - Pin 2: Left Poke -> DB25 Pin 1
  - Pin 3: Right Poke -> DB25 Pin 2
  - Pin 4: Pellet Intake -> DB25 Pin 3
  - Pin 5: LeftWithPellet -> DB25 Pin 4
  - Pin 6: RightWithPellet -> DB25 Pin 5

- **FED 2**:
  - Pin 7: Left Poke -> DB25 Pin 6
  - Pin 8: Right Poke -> DB25 Pin 7
  - Pin 9: Pellet Intake -> DB25 Pin 8
  - Pin 10: LeftWithPellet -> DB25 Pin 9
  - Pin 11: RightWithPellet -> DB25 Pin 10

- **Grounds**:
  - Connect all grounds to DB25 Pins 21-25.

### Step 4: Connecting to the TDT System

1. **DB25 to TDT**:
   - Connect the DB25 connector to the TDT system's input port (if available).
   - If the TDT system doesn't have a DB25 input, use a DB25 to BNC breakout cable or panel to convert the DB25 signals back to BNC for connection to the TDT system.

2. **Testing**:
   - Test the setup by triggering events from the FEDs and ensuring that the corresponding signals are correctly received by the TDT system.

### Conclusion:
By using a DB25 connector, you can consolidate many signals into a single, organized connection, making it easier to manage multiple FEDs. The Arduino Mega’s additional pins allow you to scale the system, and the DB25 connector simplifies the wiring to the TDT system.



### Directly Connecting Arduino to a DB25 Connector

1. **Choose the Appropriate DB25 Connector**:
   - Use a **DB25 male connector** (or female, depending on the input on your TDT system) with a breakout board or solderable connections.
   - A DB25 connector has 25 pins, which can be used to transmit multiple signals (one per Arduino pin) and a shared ground.

2. **Wiring the Arduino Pins to the DB25 Connector**:
   - **Signal Wires**: Connect each Arduino digital pin (e.g., pins 2 through 21) directly to a corresponding pin on the DB25 connector. These connections will carry the signals for each behavioral event.
   - **Ground Connection**: Connect the Arduino GND to one or more of the DB25 connector's ground pins (e.g., pins 21-25). All signal wires will share this common ground.

### Example Pin Assignment:

Assume you're connecting 4 FEDs, each with 5 events:

- **FED 1**:
  - Arduino Pin 2 (Left Poke) -> DB25 Pin 1
  - Arduino Pin 3 (Right Poke) -> DB25 Pin 2
  - Arduino Pin 4 (Pellet Intake) -> DB25 Pin 3
  - Arduino Pin 5 (LeftWithPellet) -> DB25 Pin 4
  - Arduino Pin 6 (RightWithPellet) -> DB25 Pin 5

- **FED 2**:
  - Arduino Pin 7 (Left Poke) -> DB25 Pin 6
  - Arduino Pin 8 (Right Poke) -> DB25 Pin 7
  - Arduino Pin 9 (Pellet Intake) -> DB25 Pin 8
  - Arduino Pin 10 (LeftWithPellet) -> DB25 Pin 9
  - Arduino Pin 11 (RightWithPellet) -> DB25 Pin 10

- **FED 3 and FED 4**: Continue similarly up to pin 20 or more if needed.

- **Grounds**:
  - Arduino GND -> DB25 Pins 21-25 (or any available ground pins).

### Connecting the DB25 Cable to TDT:

- **Direct DB25 Connection**:
  - If your TDT system has a DB25 input, you can directly connect the DB25 cable from the Arduino to the TDT system. This will transmit all the signals through the DB25 connector to the TDT system.

- **DB25 to BNC Breakout (if needed)**:
  - If your TDT system only accepts BNC inputs, you could use a DB25 to BNC breakout cable or panel to convert the signals from the DB25 connector to individual BNC connectors.

### Advantages of This Approach:
- **Simplified Wiring**: You avoid the complexity of multiple BNC connectors, instead routing all signals through a single, neat DB25 cable.
- **Scalability**: The DB25 connector can handle up to 24 signals (plus ground), making it easy to scale your setup for multiple FEDs.
- **Organized Setup**: This approach reduces clutter and makes it easier to manage the connections between the Arduino and the TDT system.

### Summary:
You can indeed directly connect the output pins of the Arduino and the ground pin to a DB25 connector, eliminating the need for separate BNC connectors. This approach simplifies the wiring and makes it easier to manage multiple FEDs in your setup.



# How to Use a DB25 Pigtail Cable:

1. **Purchase a DB25 Pigtail Cable**:
   - Look for a **DB25 male or female pigtail cable** with 25 individual wires. These cables have a DB25 connector on one end and 25 color-coded wires (often tinned) on the other end.

2. **Connect the Wires to Arduino**:
   - **Signal Wires**: Connect the individual wires from the DB25 pigtail to the corresponding digital pins on the Arduino Mega.
     - For example, the wire corresponding to DB25 pin 1 would connect to Arduino pin 2, DB25 pin 2 to Arduino pin 3, and so on.
   - **Ground Wires**: Connect one or more of the ground wires (typically from DB25 pins 21-25) to the Arduino's GND pin.

3. **Secure the Connections**:
   - You can connect the pigtail wires directly into the Arduino's female header slots. If needed, use jumper wires to extend or make the connections more secure.
   - Ensure that the connections are stable and properly seated to avoid disconnections during operation.

### Example Pinout for DB25 Pigtail Cable:

Assuming a DB25 male pigtail cable:

- **FED 1**:
  - DB25 Pin 1 (Wire 1): Connect to Arduino Pin 2 (Left Poke)
  - DB25 Pin 2 (Wire 2): Connect to Arduino Pin 3 (Right Poke)
  - DB25 Pin 3 (Wire 3): Connect to Arduino Pin 4 (Pellet Intake)
  - DB25 Pin 4 (Wire 4): Connect to Arduino Pin 5 (LeftWithPellet)
  - DB25 Pin 5 (Wire 5): Connect to Arduino Pin 6 (RightWithPellet)

- **FED 2**:
  - DB25 Pin 6 (Wire 6): Connect to Arduino Pin 7 (Left Poke)
  - DB25 Pin 7 (Wire 7): Connect to Arduino Pin 8 (Right Poke)
  - DB25 Pin 8 (Wire 8): Connect to Arduino Pin 9 (Pellet Intake)
  - DB25 Pin 9 (Wire 9): Connect to Arduino Pin 10 (LeftWithPellet)
  - DB25 Pin 10 (Wire 10): Connect to Arduino Pin 11 (RightWithPellet)

- **Ground**:
  - DB25 Pins 21-25 (Wires 21-25): Connect one or more to the Arduino GND pin.

### Advantages:
- **Simplified Wiring**: Directly connect the DB25 cable to the Arduino without needing an intermediate breakout board.
- **Less Clutter**: Reduces the need for additional jumper wires and connectors, making the setup cleaner.
- **Direct Connections**: Ensures that each Arduino pin is directly connected to the correct pin on the DB25 connector.

### Summary:
Using a DB25 pigtail cable with free wires allows you to connect the Arduino's digital pins directly to the DB25 connector, simplifying the wiring process. This approach is efficient and reduces the need for additional breakout boards or connectors, making your setup neater and easier to manage.